# Big Data Platforms

## PySpark Machine Learning

### MLlib applied to Wine reviews data 

**Dataset:**
https://www.kaggle.com/zynicide/wine-reviews


Copyright: 2018 [Ashish Pujari](apujari@uchicago.edu)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#create Spark session
spark = SparkSession.builder.appName('WineReviewsML').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '5g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])

#print spark configuration settings
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1550640219048'),
 ('spark.executor.id', 'driver'),
 ('spark.executor.memory', '5g'),
 ('spark.executor.cores', '4'),
 ('spark.cores.max', '4'),
 ('spark.driver.host', 'DESKTOP-RV4I2EU'),
 ('spark.app.name', 'Spark Updated Conf'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '8g'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '63064'),
 ('spark.ui.showConsoleProgress', 'true')]

## Read Data

In [3]:
df = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .csv("T:\\courses\\BigData\\data\\wine-reviews\\winemag-data_first150k.csv",inferSchema=True, header=True )

In [4]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [5]:
df2 = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .csv("T:\\courses\\BigData\\data\\wine-reviews\\winemag-data-130k-v2.csv",inferSchema=True, header=True )

In [6]:
df2.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- taster_name: string (nullable = true)
 |-- taster_twitter_handle: string (nullable = true)
 |-- title: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [7]:
#combine the two datasets
df = df.union(df2.drop("taster_name", "taster_twitter_handle", "title"))

## Data Exploration

In [8]:
df.count()

280901

In [9]:
df.show(5)

+---+-------+--------------------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+
|_c0|country|         description|         designation|points|price|      province|         region_1|         region_2|           variety|              winery|
+---+-------+--------------------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+
|  0|     US|This tremendous 1...|   Martha's Vineyard|    96|235.0|    California|      Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  1|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|110.0|Northern Spain|             Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|  2|     US|Mac Watson honors...|Special Selected ...|    96| 90.0|    California|   Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|  3|     US|This spent 20 mon...|      

In [10]:
#Count rows with missing values
df.dropna().count()

73325

##  Feature Engineering

In [11]:
from pyspark.ml.feature import QuantileDiscretizer

#High Medium Low
discretizer = QuantileDiscretizer(numBuckets=3, inputCol="points", outputCol="points_category")
df = discretizer.fit(df).transform(df)
df.show(3)

+---+-------+--------------------+--------------------+------+-----+--------------+--------------+--------+------------------+--------------------+---------------+
|_c0|country|         description|         designation|points|price|      province|      region_1|region_2|           variety|              winery|points_category|
+---+-------+--------------------+--------------------+------+-----+--------------+--------------+--------+------------------+--------------------+---------------+
|  0|     US|This tremendous 1...|   Martha's Vineyard|    96|235.0|    California|   Napa Valley|    Napa|Cabernet Sauvignon|               Heitz|            2.0|
|  1|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|110.0|Northern Spain|          Toro|    null|     Tinta de Toro|Bodega Carmen Rod...|            2.0|
|  2|     US|Mac Watson honors...|Special Selected ...|    96| 90.0|    California|Knights Valley|  Sonoma|   Sauvignon Blanc|            Macauley|            2.0|
+---+-------+---

## Natural Language Processing

In [12]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

#tokenize words
tokenizer = Tokenizer(inputCol="description", outputCol="words")
df = tokenizer.transform(df)

#drop the redundant source column
df= df.drop("description")
df.show(5)

+---+-------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+---------------+--------------------+
|_c0|country|         designation|points|price|      province|         region_1|         region_2|           variety|              winery|points_category|               words|
+---+-------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+---------------+--------------------+
|  0|     US|   Martha's Vineyard|    96|235.0|    California|      Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|            2.0|[this, tremendous...|
|  1|  Spain|Carodorum Selecci...|    96|110.0|Northern Spain|             Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|            2.0|[ripe, aromas, of...|
|  2|     US|Special Selected ...|    96| 90.0|    California|   Knights Valley|           Sonoma|   Sauvignon Blanc|   

In [13]:
from pyspark.ml.feature import StopWordsRemover

#remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
df = remover.transform(df)

#drop the redundant source column
df= df.drop("words")
df.show(5)

+---+-------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+---------------+--------------------+
|_c0|country|         designation|points|price|      province|         region_1|         region_2|           variety|              winery|points_category|            filtered|
+---+-------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+---------------+--------------------+
|  0|     US|   Martha's Vineyard|    96|235.0|    California|      Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|            2.0|[tremendous, 100%...|
|  1|  Spain|Carodorum Selecci...|    96|110.0|Northern Spain|             Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|            2.0|[ripe, aromas, fi...|
|  2|     US|Special Selected ...|    96| 90.0|    California|   Knights Valley|           Sonoma|   Sauvignon Blanc|   

In [14]:
#Maps a sequence of terms to their term frequencies using the hashing trick. 
#alternatively, CountVectorizer can also be used to get term frequency vectors
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(df)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
nlpdf = idfModel.transform(featurizedData)
nlpdf.select("points_category", "features").show(10)

+---------------+--------------------+
|points_category|            features|
+---------------+--------------------+
|            2.0|(20,[0,1,3,4,5,7,...|
|            2.0|(20,[0,1,2,4,5,6,...|
|            2.0|(20,[2,3,4,6,7,8,...|
|            2.0|(20,[0,1,2,3,4,5,...|
|            2.0|(20,[0,1,2,4,5,6,...|
|            2.0|(20,[0,1,2,4,5,6,...|
|            2.0|(20,[1,2,4,5,6,7,...|
|            2.0|(20,[0,1,2,4,5,6,...|
|            2.0|(20,[0,1,2,3,4,5,...|
|            2.0|(20,[0,2,3,6,7,8,...|
+---------------+--------------------+
only showing top 10 rows



In [15]:
nlpdf.show(5)

+---+-------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+---------------+--------------------+--------------------+--------------------+
|_c0|country|         designation|points|price|      province|         region_1|         region_2|           variety|              winery|points_category|            filtered|         rawFeatures|            features|
+---+-------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+---------------+--------------------+--------------------+--------------------+
|  0|     US|   Martha's Vineyard|    96|235.0|    California|      Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|            2.0|[tremendous, 100%...|(20,[0,1,3,4,5,7,...|(20,[0,1,3,4,5,7,...|
|  1|  Spain|Carodorum Selecci...|    96|110.0|Northern Spain|             Toro|             null|     Tinta de Toro|Bodega Carm

In [16]:
#split data into train and test
splits = nlpdf.randomSplit([0.8, 0.2])
train_df = splits[0]
test_df = splits[1]

train_df.show(1)

+---+-------+-----------------+------+-----+----------+-----------+--------+------------------+------+---------------+--------------------+--------------------+--------------------+
|_c0|country|      designation|points|price|  province|   region_1|region_2|           variety|winery|points_category|            filtered|         rawFeatures|            features|
+---+-------+-----------------+------+-----+----------+-----------+--------+------------------+------+---------------+--------------------+--------------------+--------------------+
|  0|     US|Martha's Vineyard|    96|235.0|California|Napa Valley|    Napa|Cabernet Sauvignon| Heitz|            2.0|[tremendous, 100%...|(20,[0,1,3,4,5,7,...|(20,[0,1,3,4,5,7,...|
+---+-------+-----------------+------+-----+----------+-----------+--------+------------------+------+---------------+--------------------+--------------------+--------------------+
only showing top 1 row



### Logistic Regression Model

In [17]:
from pyspark.ml.classification import LogisticRegression

# Set parameters for Logistic Regression
lgr = LogisticRegression(maxIter=10, featuresCol = 'features', labelCol='points_category')

# Fit the model to the data.
lgrm = lgr.fit(train_df)

# Given a dataset, predict each point's label, and show the results.
predictions = lgrm.transform(test_df)

In [18]:
predictions.show(3)

+---+-------+--------------------+------+-----+----------+--------------------+--------+------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|_c0|country|         designation|points|price|  province|            region_1|region_2|           variety|              winery|points_category|            filtered|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+---+-------+--------------------+------+-----+----------+--------------------+--------+------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  4| France|          La Brûlade|    95| 66.0|  Provence|              Bandol|    null|Provence red blend|Domaine de la Bégude|            2.0|[top, wine, la, b...|(20,[0,1,2,4,5,6,...|(20,[0,1,2,4,5,6,...|[-1.551037

In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#print evaluation metrics
evaluator = MulticlassClassificationEvaluator(labelCol="points_category", predictionCol="prediction")

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))

0.5121946871100018
0.509725237114735


### Word2Vec

In [20]:
# Learn a mapping from words to Vectors
from pyspark.ml.feature import Word2Vec
word2Vec = Word2Vec(vectorSize=5, minCount=0, inputCol="filtered", outputCol="wordVectors")
w2VM = word2Vec.fit(df)
nlpdf = w2VM.transform(df)

In [21]:
nlpdf.select("points_category", "wordVectors").show(2, truncate=False)

+---------------+-------------------------------------------------------------------------------------------------------+
|points_category|wordVectors                                                                                            |
+---------------+-------------------------------------------------------------------------------------------------------+
|2.0            |[0.002250502888475441,-0.18227338501148752,-0.14161098287958238,0.13593680339141023,0.2529686127220177]|
|2.0            |[-0.03944722082345716,-0.4090915118013659,0.22312594962216192,0.008944106798979544,0.28818741669097253]|
+---------------+-------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [22]:
#split data into train and test
splits = nlpdf.randomSplit([0.8, 0.2])
train_df = splits[0]
test_df = splits[1]

train_df.show(1)

+---+-------+-----------------+------+-----+----------+-----------+--------+------------------+------+---------------+--------------------+--------------------+
|_c0|country|      designation|points|price|  province|   region_1|region_2|           variety|winery|points_category|            filtered|         wordVectors|
+---+-------+-----------------+------+-----+----------+-----------+--------+------------------+------+---------------+--------------------+--------------------+
|  0|     US|Martha's Vineyard|    96|235.0|California|Napa Valley|    Napa|Cabernet Sauvignon| Heitz|            2.0|[tremendous, 100%...|[0.00225050288847...|
+---+-------+-----------------+------+-----+----------+-----------+--------+------------------+------+---------------+--------------------+--------------------+
only showing top 1 row



In [23]:
# Set parameters for Logistic Regression
lgr = LogisticRegression(maxIter=10, featuresCol = 'wordVectors', labelCol='points_category')

# Fit the model to the data.
lgrm = lgr.fit(train_df)

# Given a dataset, predict each point's label, and show the results.
predictions = lgrm.transform(test_df)

In [24]:
#print evaluation metrics
evaluator = MulticlassClassificationEvaluator(labelCol="points_category", predictionCol="prediction")

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))

0.46087065031374785
0.45402426901079673


<b>Exercise</b>: <font color='red'>Fine tune the Word2vec method to improve model accuracy </font>